In [29]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')
import numpy as np
# Para la visualización 
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
# Otros objetivos
# -----------------------------------------------------------------------
import math
from itertools import combinations
# Para pruebas estadísticas
# -----------------------------------------------------------------------
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
# Para la codificación de las variables numéricas
# -----------------------------------------------------------------------
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder # para poder aplicar los métodos de OneHot, Ordinal,  Label y Target Encoder 
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder, CountEncoder
import sys
sys.path.append("../../")
import src.soporte_encoding  as se
from src.soporte_graficas import (
    visualizar_categoricas
)
from src.SupportPreProcesamiento import (
    separarar_df
)

In [30]:
df = pd.read_csv("../../datos/03_api_rent_sin_nulos.csv")

# Encoding
- Vamos a pasar todas las variables a numéricas
- Hay que saber si tienen orden o no

### Pasar las columnas a su tipo correcto

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         353 non-null    float64
 1   size          353 non-null    float64
 2   rooms         353 non-null    object 
 3   bathrooms     353 non-null    object 
 4   floor         353 non-null    object 
 5   municipality  353 non-null    object 
 6   hasLift       353 non-null    object 
dtypes: float64(2), object(5)
memory usage: 19.4+ KB


In [32]:
df["rooms"] = df["rooms"].astype("O")
df["bathrooms"] = df["bathrooms"].astype("O")

# Analicemos estadísticamente
- Dos opciones: mannwhitneyu
- Mas de dos opciones: Kruskal

No es una distribución normal

In [33]:
numericas,categoricas = separarar_df(df)

In [34]:
# instanciamos la clase de Asunciones, para poder decir qué método usar para el análisis estadístico
asunciones = se.Asunciones(dataframe = df, columna_numerica = "price")
print("\n-------------\n")


for categoria in categoricas:   

    print(f"Estamos analizando la variable {categoria.upper()}")
    
    asunciones.identificar_normalidad(metodo = "kolmogorov") 

    # comprobamos la homogeneidad de varianzas
    asunciones.identificar_homogeneidad(columna_categorica = categoria)

    # instanciamos la clase para evaluar si hay diferencias entre los distintos grupos de las variables categóricas
    test = se.TestEstadisticos(df, "price", categoria)
    test.run_all_tests()
    print("\n###########################\n")


-------------

Estamos analizando la variable ROOMS
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable rooms las varianzas son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['1 habitacion', '2 habitaciones', '3 habitaciones', 'sin habitaciones', '4 habitaciones']

Realizando test de Kruskal...
Estadístico de prueba: 6.997445029790361
Valor p: 0.13602330549397834
No hay evidencia suficiente para concluir que hay una diferencia significativa.
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,15.3159,0.2789,-5.9159,36.5476,1 habitacion-2 habitaciones
1,22.5779,0.1712,-5.1881,50.3438,1 habitacion-3 habitaciones
2,30.6804,0.9113,-67.1878,128.5487,1 habitacion-4 habitaciones
3,8.2043,0.9324,-20.1500,36.5585,1 habitacion-sin habitaciones
4,7.2620,0.9511,-20.2748,34.7988,2 habitaciones-3 habitaciones
5,15.3646,0.9928,-82.4389,113.1680,2 habitaciones-4 habitaciones
6,-7.1116,0.9579,-35.2415,21.0183,2 habitaciones-sin habitaciones
7,8.1026,0.9994,-91.3242,107.5293,3 habitaciones-4 habitaciones
8,-14.3736,0.7616,-47.7121,18.9648,3 habitaciones-sin habitaciones
9,-22.4762,0.9720,-122.0688,77.1164,4 habitaciones-sin habitaciones



###########################

Estamos analizando la variable BATHROOMS
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable bathrooms las varianzas NO son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['1 aseo', '2 aseos', '3 aseos']

Realizando test de Kruskal...
Estadístico de prueba: 2.17914787981004
Valor p: 0.3363597726585303
No hay evidencia suficiente para concluir que hay una diferencia significativa.
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,-16.9578,0.2492,-41.9852,8.0696,1 aseo-2 aseos
1,55.4476,0.6376,-88.7934,199.6886,1 aseo-3 aseos
2,72.4054,0.4733,-73.5403,218.3511,2 aseos-3 aseos



###########################

Estamos analizando la variable FLOOR
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable floor las varianzas son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['altos', 'bajos', 'desconocido']

Realizando test de Kruskal...
Estadístico de prueba: 9.900869359787336
Valor p: 0.007080330582708134
Hay una diferencia significativa entre los grupos
Los resultados del test de Tukey son: 



...


###########################

Estamos analizando la variable MUNICIPALITY
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable municipality las varianzas NO son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['Clase Media-Alta', 'Clase Alta', 'Clase Obrera', 'Clase Media']

Realizando test de Kruskal...
Estadístico de prueba: 13.676646647757684
Valor p: 0.003380025836555914
Hay una diferencia significativa entre los grupos
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,-43.8450,0.0766,-90.7600,3.0701,Clase Alta-Clase Media
1,-19.5949,0.6934,-65.8293,26.6395,Clase Alta-Clase Media-Alta
2,-39.1961,0.3115,-97.8061,19.4139,Clase Alta-Clase Obrera
3,24.2501,0.0025,6.6081,41.8921,Clase Media-Clase Media-Alta
4,4.6489,0.9907,-35.4604,44.7582,Clase Media-Clase Obrera
5,-19.6012,0.5717,-58.9122,19.7098,Clase Media-Alta-Clase Obrera



###########################

Estamos analizando la variable HASLIFT
Para la columna price, los datos no siguen una distribución normal según el test de Kolmogorov-Smirnov.
En la variable hasLift las varianzas son homogéneas entre grupos.
Generando grupos...
Grupos generados: ['tiene ascensor', 'no tiene ascensor', 'desconocido']

Realizando test de Kruskal...
Estadístico de prueba: 5.798984817212032
Valor p: 0.05505115645891607
No hay evidencia suficiente para concluir que hay una diferencia significativa.
Los resultados del test de Tukey son: 



,meandiff,p-adj,lower,upper,group_diff
0,24.0599,0.4173,-20.7870,68.9067,desconocido-no tiene ascensor
1,31.6032,0.2211,-13.1327,76.3390,desconocido-tiene ascensor
2,7.5433,0.4906,-8.0419,23.1285,no tiene ascensor-tiene ascensor



###########################



# Resultados
### Variables ordinales
- floor
- municipality
### Variables Nominales
- propertyType
- rooms
- bathrooms
- hasLift

# Tenemos Ordinales y Nominales
- Las ordinales las gestionaremos con Target Encoding, que codifica con la media del objetivo para la categoría
- Para las nominales usaremos one-Hot Encoding

In [35]:
df.sample()

,price,size,rooms,bathrooms,floor,municipality,hasLift
333,700.0,55.0,1 habitacion,1 aseo,bajos,Clase Media-Alta,tiene ascensor


# Ordinales

In [36]:
cols_ordinales = ["floor","municipality"]

### Target Encoder

In [37]:
target_encoder = TargetEncoder(cols=cols_ordinales)
df = target_encoder.fit_transform(df,df[["price"]])

In [38]:
df.sample()

,price,size,rooms,bathrooms,floor,municipality,hasLift
197,750.0,25.0,sin habitaciones,1 aseo,702.264503,702.071795,tiene ascensor


# Nominales

In [39]:
cols_nominales = ["rooms","bathrooms","hasLift"]

### OneHotEncoder

In [40]:
encoder = OneHotEncoder(categories='auto', 
                        drop=None, 
                        sparse_output=True, 
                        dtype='float', 
                        handle_unknown='error')

# Ajustar el codificador a los datos y transformarlos
encoder_trans = encoder.fit_transform(df[cols_nominales])

# lo siguiente que hacemos es convertir el objeto devuelto por el fit_transform a array para poder verlo
encoder_array = encoder_trans.toarray()

# usaremos el método '.get_feature_names_out()' para extraer el nombre de las columnas
nombre_columnas = encoder.get_feature_names_out()

# creamos un DataFrame con los resultados obtenidos de la transformación
encoder_df = pd.DataFrame(encoder_array, columns = nombre_columnas)

# concatenamos estos resultados con el DataFrame original
df = pd.concat([df, encoder_df], axis = 1)

In [41]:
df.sample()

,price,size,rooms,bathrooms,floor,municipality,hasLift,rooms_1 habitacion,rooms_2 habitaciones,rooms_3 habitaciones,rooms_4 habitaciones,rooms_sin habitaciones,bathrooms_1 aseo,bathrooms_2 aseos,bathrooms_3 aseos,hasLift_desconocido,hasLift_no tiene ascensor,hasLift_tiene ascensor
29,650.0,62.0,2 habitaciones,1 aseo,686.923636,702.071795,no tiene ascensor,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


# Eliminar columnas nominales
- Ya tenemos las columnas generadas y ya no nos aportan información

In [42]:
df.drop(columns=cols_nominales,inplace=True)

In [43]:
df.sample()

,price,size,floor,municipality,rooms_1 habitacion,rooms_2 habitaciones,rooms_3 habitaciones,rooms_4 habitaciones,rooms_sin habitaciones,bathrooms_1 aseo,bathrooms_2 aseos,bathrooms_3 aseos,hasLift_desconocido,hasLift_no tiene ascensor,hasLift_tiene ascensor
38,700.0,40.0,702.264503,702.071795,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


# Guardamos
- Nos vamos a feature scaling con este df

In [44]:
df.to_csv("../../datos/04_rent_target_onehot_encoded.csv",index=False)